In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio

In [ ]:
dataObjs = curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)

In [29]:
allModels = []
tickersSeen = []
for item in dataObjs:
    try:
        if item["IS_PROFITABILITY SLIPPAGE"] > 0.505 and item["IS_ANNUALIZED RETURN"] > 0.15:
            model = item["model"]
            print(model.targetTicker, model.getHash(), item["IS_SHARPE SLIPPAGE"], item["IS_SHARPE DIFFERENCE SLIPPAGE"], item["IS_BETA"])
            allModels.append(model)
            if model.targetTicker not in tickersSeen:
                tickersSeen.append(model.targetTicker)
    except:
        continue
        

XBI d3790181229ad72221d52af2f5c194a4502d057ff65102cb1722fa0e 0.7016792457881373 0.0007479863352591876 0.4870775187704768
XBI 74d57b245c65edc16ba9543db1532ae054cbe2351c87d471d3efcb96 0.7857980316611682 0.003950267828858323 0.46332945928677816
IYT 24e04b9013aa07c5486e1dc8d433e1e76f1c98a547a780fb024657bf 0.8119993502473207 0.004460887515734702 0.3454585848152516
XBI c1d65e48a893b0e4da27a94556627441b0041a1bee9ead69d652160f 0.7141920072174859 0.009536833474352302 0.5964559370685579
IBB a76437cd8386d90fa5a74840ece0fa8b78f1c9bf1c0b8f55b4c2b57b 0.7359168690750022 0.012233453667827021 0.4271098393393847
XBI 7c3371dc5ada343504f31ff99a9aa990085e6e8696b41e303d220b03 0.7230515808732165 0.020026300734221847 0.5977221077627428
XRT 29a563f5e88628c36692b198c4fc861621f7a8f12f67c0c842e88fcb 0.9302900630459066 0.02451843467972592 0.411165639927419
SOXX bc06268875f6c68a03acdaa4ed0ee8b08535db839e75bfd9f60163ba 0.8171455417168544 0.028768690105633432 0.21613550753474767
XBI b00f49b7e171f804314101b39b4aec448f

In [30]:
len(allModels)

147

In [31]:
len(tickersSeen)

19

In [32]:
import random
factorToTrade = "VTI"#tickersSeen[random.randint(0, len(tickersSeen) - 1)]
factorToTrade

'VTI'

In [35]:
import importlib
importlib.reload(dataAck)
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


['XBI', 'IYT', 'IBB', 'XRT', 'SOXX', 'IWN', 'IWM', 'IYR', 'XHB', 'VNQ', 'IJR', 'XLI', 'VGT', 'KRE', 'IYF', 'XLB', 'SLV', 'KBE', 'XME', 'VTI']


IndexError: list index out of range

In [9]:
cleanedReturns = modelReturns.dropna()
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.dropna()
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item

In [10]:
def historicalWeightsToTickerAllocations(historicalWeights, algorithmPredictions, modelsInPortfolio):
    aggregatePredictions = algorithmPredictions.dropna()
    allocationsToStore = []
    historicalAllocations = None
    scaledHistoricalAllocations = None
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.targetTicker not in netPosition:
                netPosition[model.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.targetTicker] += weights[model.getHash()] * aggregatePredictions.loc[historicalWeights.index[i]][model.getHash()]
        thisDf = pd.DataFrame([netPosition], index=[historicalWeights.index[i]])
        if historicalAllocations is None:
            historicalAllocations = thisDf
        else:
            historicalAllocations = pd.concat([historicalAllocations, thisDf])
        
        totalCapitalUsed = sum([abs(netPosition[ticker]) for ticker in netPosition])
        scaledNetPosition = {}
        for ticker in netPosition:
            scaledNetPosition[ticker] = netPosition[ticker] * 1.0/totalCapitalUsed
        
        thisDf = pd.DataFrame([scaledNetPosition], index=[historicalWeights.index[i]])
        if scaledHistoricalAllocations is None:
            scaledHistoricalAllocations = thisDf
        else:
            scaledHistoricalAllocations = pd.concat([scaledHistoricalAllocations, thisDf])
    
    return historicalAllocations, scaledHistoricalAllocations
            
            

In [11]:
import empyrical
def getLimitedDataForPortfolio(historicalWeights, historicalPredictions, modelsUsed, factorToTrade, joinedData):
    
    normalTickerAllocationsTable, scaledTickerAllocationsTable = historicalWeightsToTickerAllocations(historicalWeights, historicalPredictions, modelsUsed)
    
    tickerAllocationsTable = scaledTickerAllocationsTable
    print(tickerAllocationsTable)
    rawTickerPerformance = portfolioGeneration.calculatePerformanceForTable(tickerAllocationsTable, tickerAllocationsTable.columns, joinedData)

    rawAlgoPerformance = pd.DataFrame(rawTickerPerformance.apply(lambda x:sum(x), axis=1), columns=["Algo Return Without Commissions"])

    tickerPerformance, algoPerformance, algoTransactionCost =  portfolioGeneration.calculatePerformanceForAllocations(tickerAllocationsTable, joinedData)

    benchmark = factorToTrade
    factorReturn = dataAck.getDailyFactorReturn(benchmark, joinedData)
    factorReturn.columns = ["Factor Return (" + benchmark + ")"]
    algoPerformance.columns = ["Algo Return"]
    
    tickersUsed = []
    for mod in modelsUsed:
        tickersUsed.append(mod.targetTicker)
    
    for ticker in tickersUsed:
        thisFactorReturn = dataAck.getDailyFactorReturn(ticker, joinedData)
        thisFactorReturn.columns = ["Factor Return (" + ticker + ")"]
        alpha, beta = empyrical.alpha_beta(algoPerformance, thisFactorReturn)
        print(ticker, beta)

    alpha, beta = empyrical.alpha_beta(algoPerformance, factorReturn)
    sharpe_difference = empyrical.sharpe_ratio(algoPerformance) - empyrical.sharpe_ratio(factorReturn)
    annualizedReturn = empyrical.annual_return(algoPerformance)[0]
    annualizedVolatility = empyrical.annual_volatility(algoPerformance)
    
    ##AUTOMATICALLY TAKES SLIPPAGE INTO ACCOUNT
    return {
        "benchmark":factorToTrade,
        "alpha":alpha,
        "beta":beta,
        "sharpe difference":sharpe_difference,
        "annualizedReturn":annualizedReturn,
        "annualizedVolatility":annualizedVolatility,
        "sharpe":empyrical.sharpe_ratio(algoPerformance),
        "free return":annualizedReturn - annualizedVolatility
    }
    


In [20]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(15, len(algoColumns)), replace= False)


In [21]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            
            cov = (aggregateReturns[i-windowSize:i]).cov()
        try:
            weights = hrp.getHRP(cov, corr)
        #     display(weights)
        #     display(aggregateReturns[i+windowSize:i+windowSize+1])
            todayReturn = aggregateReturns[i:i+1] * weights
        #     display(todayReturn)
            sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
            hrpReturns = pd.concat([hrpReturns, sumReturn])
            thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
            historicalWeights = pd.concat([historicalWeights, thisWeights])
        except:
            print("FAILED:",i)
        i += 1
    return hrpReturns, historicalWeights

In [22]:
def storeDiscoveredPortfolio(models, portfolioType, benchmark, IS_DATA, OOS_DATA):
    description = "AUTO GENERATED"
    portfolioType = "HRP FULL"
    seenTickers = []
    
    allHashes = []
    for model in models:
        allHashes.append(model.getHash())
        if model.targetTicker not in seenTickers:
            seenTickers.append(model.targetTicker)
        
    ##SORT SO ENSURE SAME PORTFOLIO NOT CREATED TWICE
    allHashes = sorted(allHashes)
    
    portfolioString = str(allHashes) + benchmark + description + portfolioType
    portfolioHash = hashlib.sha224(portfolioString.encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.discoveredPortfolioModels, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark,
                "portfolioType":portfolioType,
                "startedTrading":curveTreeDB.getToday()
            }
            
            for k in IS_DATA:
                toUpload["IS_"+ k] = IS_DATA[k]
            
            for k in OOS_DATA:
                toUpload["OOS_"+ k] = OOS_DATA[k]
                
            for ticker in seenTickers:
                toUpload[ticker] = True
            
            toUpload["TICKERS TRADED"] = len(seenTickers)
            
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.discoveredPortfolios, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            return portfolioHash
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [23]:
def getWeightingForAlgos(allModels, columns):
    countPerTicker = {}
    hashes = {}
    for mod in allModels:
        hashes[mod.getHash()] = mod.targetTicker
        if mod.targetTicker not in countPerTicker:
            countPerTicker[mod.targetTicker] = 0.0
        countPerTicker[mod.targetTicker] += 1.0
    weightsToSend = []
    for col in columns:
        weightsToSend.append(1.0/countPerTicker[hashes[col]])
        
    return [item/sum(weightsToSend) for item in weightsToSend]

In [24]:
def binarizeReturns(returnArr):
    newArr = []
    for item in returnArr:
        if item > 0.0:
            newArr.append(1.0)
        elif item < 0.0:
            newArr.append(-1.0)
        else:
            newArr.append(0.0)
    return newArr

In [25]:
def performPortfolioPerformanceEstimation(historicalPredictions, historicalReturns, factorToTrade, portfolioType, hashToModel, joinedData):
    returnWindows = [(0, historicalReturns[:450]), (450, historicalReturns)]
    historicalWeights = None
    for selectedReturns in returnWindows:
        startIndex = selectedReturns[0]
        returnWindow = selectedReturns[1]
        weightsSeen = None
        if portfolioType == "HRP FULL":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=False)
        elif portfolioType == "HRP BINARY":
            hrpReturns, weightsSeen = produceHRPPredictions(pd.DataFrame(returnWindow.apply(lambda x:binarizeReturns(x), axis=1)),\
                    126, startIndex=max(startIndex, 126), maxWindowSize=False)
        elif portfolioType == "HRP WINDOW":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=True)
        elif portfolioType == "EW":
            weightsSeen = pd.DataFrame(returnWindow[startIndex:].apply(lambda x: [1.0/len(x) for item in x], axis=1), columns=returnWindow.columns.values)
        elif portfolioType == "EW By Ticker":
            weightArray = getWeightingForAlgos(allModels, returnWindow.columns)
            weightsSeen = pd.DataFrame(returnWindow[startIndex:].apply(lambda x: weightArray, axis=1), columns=returnWindow.columns.values)
            
        
        
        if historicalWeights is None:
            historicalWeights = weightsSeen
        else:
            historicalWeights = pd.concat([historicalWeights, weightsSeen])
        
        modelsUsed = []

        tickersSeen = {}

        for modelHash in historicalPredictions.columns:
            thisModel = hashToModel[modelHash]
            modelsUsed.append(thisModel)
        if startIndex == 0:
            scaledStats = getLimitedDataForPortfolio(historicalWeights,\
                                    historicalPredictions, modelsUsed, factorToTrade, joinedData)
            print(scaledStats)
            if scaledStats["sharpe difference"] < 0.0 or scaledStats["annualizedReturn"] < scaledStats["annualizedVolatility"]:
                return None, None
    trainStats = getLimitedDataForPortfolio(historicalWeights[:-252], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    testStats = getLimitedDataForPortfolio(historicalWeights[-252:], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    
    if trainStats["sharpe difference"] > 0.0 and trainStats["annualizedReturn"] > trainStats["annualizedVolatility"]:
        print(trainStats, testStats)
        storeDiscoveredPortfolio(modelsUsed, portfolioType, factorToTrade, trainStats, testStats)
    else:
        print("FAILED", trainStats)
    

In [26]:
## MP RUN  
types = ["HRP BINARY", "EW", "HRP WINDOW", "HRP FULL", "EW By Ticker"]
def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        factorToTrade = "VTI"#hashToModel[selectedAlgorithms[random.randint(0, len(selectedAlgorithms) - 1)]].targetTicker
        
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
            
        portfolioType = types[random.randint(0, len(types) - 1)]
        print(factorToTrade, len(selectedAlgorithms), portfolioType)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, portfolioType, hashToModel, joinedData))
        p.start()
        runningP.append(p)

# In[ ]:


In [27]:
print("STARTING GENERATION")

##REMOVE BREAK TO DO FULL AUTO
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=0)

STARTING GENERATION
VTI 29 EW By Ticker
                 IBB       IVW       IWM       IWN       IYF       IYR  \
2009-08-27 -0.000413 -0.135208  0.210714  0.098333  0.045655  0.000000   
2009-08-28  0.011797 -0.123869  0.265433  0.082579  0.038340  0.000000   
2009-10-09 -0.002905 -0.135043 -0.092601 -0.014405  0.027780  0.033611   
2009-10-12 -0.013121 -0.169154  0.000000  0.017349  0.033459  0.034698   
2009-10-13 -0.006807 -0.195288  0.123992  0.057863  0.037198  0.035361   
2009-10-14  0.017062 -0.228418  0.145027  0.090239  0.043508  0.033840   
2009-10-15  0.039643 -0.294841  0.000000  0.151201  0.064800  0.035280   
2009-10-16  0.028088 -0.177249 -0.217040  0.118166  0.036173  0.014067   
2009-10-19  0.039199 -0.158508 -0.194092  0.135864  0.029114  0.012580   
2009-10-20  0.032348 -0.131228 -0.224963  0.116647  0.024996  0.005832   
2009-10-21  0.036886 -0.114011 -0.244309  0.114011  0.018323  0.001583   
2009-10-22  0.032577 -0.120308 -0.229159  0.106941  0.015277 -0.001485  

KeyboardInterrupt: 